### Данные Москвы и Московской области с сайта "Реформа ЖКХ"

В данном отчете представлен код для скачивания данных по домам Москвы и Московской области с сайта [Реформа ЖКХ]('https://www.reformagkh.ru'). Затем производится объединение в общий `DataFrame`. Из-за недоступности сайта все необходимые данные не удалось загрузить. Представлено несколько подходов, для последовательного скачивания с помощью `selenium` и `PhantomJS`, а также вариант в многопоточном исполнении с помощью модуля `multithreading`. Далее представлен блок с импортами, а далее с основными операциями.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import itertools as it
from multiprocessing.pool import ThreadPool as Pool

# user defined modules and functions
from get_data_from_form import get_keys_values, get_kadastr_number, get_address_anketa
from get_links_functions import (get_link_houses, get_num_pages, 
    get_list_key_value_for_area, get_keys_values_for_area_single)

# seleinum imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [23]:
browser = webdriver.PhantomJS()

url_pref_house = 'https://www.reformagkh.ru'
url_prefix = 'https://www.reformagkh.ru/myhouse'
url_mos_area = url_prefix + '?tid=2281126'
url_mos = url_prefix + 'tid=2280999'

urls = [url_mos, url_mos_area]

multithread = False

if multithread:
    num_workers = 4
    pool = Pool(num_workers)

for url in urls:
    r = requests.get(url)
    s = BeautifulSoup(r.text, 'lxml')
    list_full_keys = []
    list_full_values = []
    list_full_names = []
    link_areas = s.find('table', class_='col_list tree').find_all('a')
    for link_area in link_areas:
        start_link_area = url_prefix + link_area.attrs['href']
        print(link_area.text)
        r1 = requests.get(start_link_area)
        s1 = BeautifulSoup(r1.text, 'lxml')
        list_link_houses = []

        district = s1.find('table', class_='col_list tree') is not None
        if district:
            table_regions = s1.find_all('table', class_='col_list tree')
            link_regions = []
            for table_region in table_regions:
                link_regions.extend(table_region.find_all('a'))                

        else:
            # make list ling_regions with single link_area
            link_regions = [link_area]

        for link_region in link_regions:
            link_houses = get_link_houses(browser, link_region, district=district)
            if multithread:
                list_keys, list_values = pool.map(
                    get_keys_values_for_area_single, link_houses)
            else:
                list_keys, list_values = get_list_key_value_for_area(link_houses)

            list_full_keys.extend(list_keys)
            list_full_values.extend(list_full_values)

            num_pages = get_num_pages(link_region)
            list_full_names.extend(it.repeat(link_region.text), times=num_pages)    

Объединение в единый `DataFrame`. Добавление столбца с названием регионов из текстов ссылок.

In [ ]:
df = pd.DataFrame(list_full_values, columns=list_full_keys[0])
names_series = pd.Series(list_full_names, name='region_name')
df = pd.concat([name_series, df], axis=1)

Далее представлены несколько `DataFrame`, которые удалось извлечь до недоступности сайта с помощью функций из модуля `get_data_from_form`.

In [599]:
pd.DataFrame(list_values, columns=list_keys[0])

,"Общая площадь дома, кв.м",Наибольшее количество этажей,Год ввода в эксплуатацию,Последнее изменение анкеты,Дата начала обслуживания дома,"Количество этажей:наибольшее, ед.","Количество этажей:наименьшее, ед.","Количество помещений, в том числе:","Количество помещений, в том числе:жилых, ед.","Количество помещений, в том числе:нежилых, ед.",...,"Серия, тип постройки здания",Тип дома,Способ формирования фонда капитального ремонта,Дом признан аварийным,"Количество подъездов, ед.","Количество лифтов, ед.",Класс энергетической эффективности,Дополнительная информация,Кадастровый номер,Анкета дома
0,6 483.50,9,2000,18.12.2015 в 10:53,01.07.2015,9,9,Не заполнено,Не заполнено,Не заполнено,...,Не заполнено,Многоквартирный дом,На счете регионального оператора,Нет,2,2,Не заполнено,Не заполнено,Нет данных,"обл. Московская, п. Власиха, ул. Заозерная, д...."
1,7 464.00,9,1991,18.12.2015 в 10:54,01.07.2015,9,9,Не заполнено,Не заполнено,Не заполнено,...,Не заполнено,Многоквартирный дом,На счете регионального оператора,Нет,3,3,Не заполнено,Не заполнено,Нет данных,"обл. Московская, п. Власиха, ул. Заозерная, д...."


In [539]:
df = pd.DataFrame(list(zip(keys, values)))
df

,0,1
0,"Общая площадь дома, кв.м",6 483.50
1,Наибольшее количество этажей,9
2,Год ввода в эксплуатацию,2000
3,Последнее изменение анкеты,18.12.2015 в 10:53
4,Дата начала обслуживания дома,01.07.2015
5,"Количество этажей:наибольшее, ед.",9
6,"Количество этажей:наименьшее, ед.",9
7,"Количество помещений, в том числе:",Не заполнено
8,"Количество помещений, в том числе:жилых, ед.",Не заполнено
9,"Количество помещений, в том числе:нежилых, ед.",Не заполнено


### Выводы

На данный момент реализованы функции с обработкой формы конкретного дома и проверена на нескольких разных формах. Также реализованы функции с рекурсивным обходом домов Москвы и Московской области, но полностью не протестированы и данные не получены ввиду недоступности сайти "Реформа ЖКХ".